In [ ]:
pip install deepgram-sdk


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.3/130.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.1/164.1 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.1 MB/s eta 0:00:00


Install Sounddevice and SciPy


In [ ]:
pip install deepgram-sdk sounddevice scipy


OSError: PortAudio library not found

In [ ]:
!pip install portaudio
!apt-get install libportaudio2

ERROR: Could not find a version that satisfies the requirement portaudio (from versions: none)
ERROR: No matching distribution found for portaudio
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  libportaudio2
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 65.3 kB of archives.
After this operation, 223 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudio2 amd64 19.6.0-1.1 [65.3 kB]
Fetched 65.3 kB in 1s (103 kB/s)
Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 123621 files and directories currently installed.)
Preparing to unpack .../libportaudio2_19.6.0-1.1_amd64.deb ...
Unpacking libportaudio2:amd64 (19.6.0-1.1) ...
Setting up libportaudio2:amd64 (19.6.0-1.1) ...
Processing triggers for libc-bin (2.35-0ubuntu3.4) ...
/sbin/ldconfig.real: /usr/local/lib/libtbb.so.12

In [ ]:
import asyncio
import sounddevice as sd
import websockets
import json
from deepgram import Deepgram

# Your Deepgram API key
DEEPGRAM_API_KEY = 'a983f9ca4ed091a6b86242b903a2c64de3e75bec'

# Deepgram endpoint for real-time transcription
DEEPGRAM_REALTIME_ENDPOINT = "wss://api.deepgram.com/v1/listen"

# File to save the transcription
TEXT_FILE_PATH = "transcription.txt"

# Audio settings
SAMPLE_RATE = 16000  # Deepgram expects 16kHz audio for real-time
CHANNELS = 1  # Mono channel for simplicity
DURATION = 10  # Record for 10 seconds

# Initialize Deepgram SDK
dg_client = Deepgram(DEEPGRAM_API_KEY)

# Function to process audio and send to Deepgram for real-time transcription
async def transcribe_realtime():
    async with websockets.connect(
        f"{DEEPGRAM_REALTIME_ENDPOINT}?access_token={DEEPGRAM_API_KEY}&punctuate=true"
    ) as websocket:

        # Function to handle incoming transcription messages from Deepgram
        async def receive_transcription():
            while True:
                message = await websocket.recv()
                data = json.loads(message)

                # Extract transcription result
                if "channel" in data:
                    transcript = data["channel"]["alternatives"][0]["transcript"]
                    if transcript:
                        print("Transcription: ", transcript)
                        # Save transcription to text file
                        with open(TEXT_FILE_PATH, "a") as f:
                            f.write(transcript + "\n")

        # Start receiving transcription data asynchronously
        receive_task = asyncio.create_task(receive_transcription())

        # Function to send audio data to Deepgram
        def callback(indata, frames, time, status):
            if status:
                print(status, flush=True)
            asyncio.run_coroutine_threadsafe(websocket.send(indata), loop)

        # Open a stream to record audio
        with sd.InputStream(samplerate=SAMPLE_RATE, channels=CHANNELS, dtype='int16', callback=callback):
            print(f"Recording for {DURATION} seconds...")
            sd.sleep(int(DURATION * 1000))  # Record for the specified duration

        # Stop receiving transcription data
        await websocket.close()

        # Wait until receive task completes
        await receive_task

# Main function to start recording and transcription
def main():
    loop = asyncio.get_event_loop()
    loop.run_until_complete(transcribe_realtime())

if __name__ == "__main__":
    main()


Exception: 
            FATAL ERROR:
            You are attempting to instantiate a Deepgram object, which is no longer a class in version 3 of this SDK.

            To fix this issue:
                1. You need to revert to the previous version 2 of the SDK: pip install deepgram-sdk==2.12.0
                2. or, update your application's code to use version 3 of this SDK. See the README for more information.

            Things to consider:

                - This Version 3 of the SDK requires Python 3.10 or higher.
                  Older versions (3.9 and lower) of Python are nearing end-of-life: https://devguide.python.org/versions/
                  Understand the risks of using a version of Python nearing EOL.

                - Version 2 of the SDK will receive maintenance updates in the form of security fixes only.
                  No new features will be added to version 2 of the SDK.
            

In [ ]:
import asyncio
import sounddevice as sd
import websockets
import json
from deepgram import DeepgramClient # Changed import

# Your Deepgram API key
DEEPGRAM_API_KEY = 'a983f9ca4ed091a6b86242b903a2c64de3e75bec'

# Deepgram endpoint for real-time transcription
DEEPGRAM_REALTIME_ENDPOINT = "wss://api.deepgram.com/v1/listen"

# File to save the transcription
TEXT_FILE_PATH = "transcription.txt"

# Audio settings
SAMPLE_RATE = 16000  # Deepgram expects 16kHz audio for real-time
CHANNELS = 1  # Mono channel for simplicity
DURATION = 10  # Record for 10 seconds

# Initialize Deepgram SDK
dg_client = DeepgramClient(api_key=DEEPGRAM_API_KEY) # Changed instantiation

# Function to process audio and send to Deepgram for real-time transcription
dg_client = Deepgram(DEEPGRAM_API_KEY)

# Function to process audio and send to Deepgram for real-time transcription
async def transcribe_realtime():
    async with websockets.connect(
        f"{DEEPGRAM_REALTIME_ENDPOINT}?access_token={DEEPGRAM_API_KEY}&punctuate=true"
    ) as websocket:

        # Function to handle incoming transcription messages from Deepgram
        async def receive_transcription():
            while True:
                message = await websocket.recv()
                data = json.loads(message)

                # Extract transcription result
                if "channel" in data:
                    transcript = data["channel"]["alternatives"][0]["transcript"]
                    if transcript:
                        print("Transcription: ", transcript)
                        # Save transcription to text file
                        with open(TEXT_FILE_PATH, "a") as f:
                            f.write(transcript + "\n")

        # Start receiving transcription data asynchronously
        receive_task = asyncio.create_task(receive_transcription())

        # Function to send audio data to Deepgram
        def callback(indata, frames, time, status):
            if status:
                print(status, flush=True)
            asyncio.run_coroutine_threadsafe(websocket.send(indata), loop)

        # Open a stream to record audio
        with sd.InputStream(samplerate=SAMPLE_RATE, channels=CHANNELS, dtype='int16', callback=callback):
            print(f"Recording for {DURATION} seconds...")
            sd.sleep(int(DURATION * 1000))  # Record for the specified duration

        # Stop receiving transcription data
        await websocket.close()

        # Wait until receive task completes
        await receive_task

# Main function to start recording and transcription
def main():
    loop = asyncio.get_event_loop()
    loop.run_until_complete(transcribe_realtime())

if __name__ == "__main__":
    main()


Exception: 
            FATAL ERROR:
            You are attempting to instantiate a Deepgram object, which is no longer a class in version 3 of this SDK.

            To fix this issue:
                1. You need to revert to the previous version 2 of the SDK: pip install deepgram-sdk==2.12.0
                2. or, update your application's code to use version 3 of this SDK. See the README for more information.

            Things to consider:

                - This Version 3 of the SDK requires Python 3.10 or higher.
                  Older versions (3.9 and lower) of Python are nearing end-of-life: https://devguide.python.org/versions/
                  Understand the risks of using a version of Python nearing EOL.

                - Version 2 of the SDK will receive maintenance updates in the form of security fixes only.
                  No new features will be added to version 2 of the SDK.
            

Saving download (1).wav to download (1).wav


In [ ]:
import asyncio
from deepgram import Deepgram

# Your Deepgram API key
DEEPGRAM_API_KEY = 'a983f9ca4ed091a6b86242b903a2c64de3e75bec'

# Initialize Deepgram client
dg_client = Deepgram(DEEPGRAM_API_KEY)

# File to save the transcription
TEXT_FILE_PATH = "transcription.txt"

# Asynchronous function to transcribe the audio
async def transcribe_audio(filename):
    with open(filename, 'rb') as audio:
        source = {'buffer': audio, 'mimetype': 'audio/wav'}

        # Call Deepgram API for transcription
        response = await dg_client.transcription.prerecorded(source, {'punctuate': True})

        # Extract the transcription
        transcript = response['results']['channels'][0]['alternatives'][0]['transcript']
        print("Transcription: ", transcript)

        # Save transcription to text file
        with open(TEXT_FILE_PATH, 'w') as f:
            f.write(transcript)
        print(f"Transcription saved to {TEXT_FILE_PATH}")

# Run the transcription function
asyncio.run(transcribe_audio(audio_file_path))


Exception: 
            FATAL ERROR:
            You are attempting to instantiate a Deepgram object, which is no longer a class in version 3 of this SDK.

            To fix this issue:
                1. You need to revert to the previous version 2 of the SDK: pip install deepgram-sdk==2.12.0
                2. or, update your application's code to use version 3 of this SDK. See the README for more information.

            Things to consider:

                - This Version 3 of the SDK requires Python 3.10 or higher.
                  Older versions (3.9 and lower) of Python are nearing end-of-life: https://devguide.python.org/versions/
                  Understand the risks of using a version of Python nearing EOL.

                - Version 2 of the SDK will receive maintenance updates in the form of security fixes only.
                  No new features will be added to version 2 of the SDK.
            

In [ ]:
import asyncio
from deepgram import DeepgramClient # Use DeepgramClient

# Your Deepgram API key
DEEPGRAM_API_KEY = 'a983f9ca4ed091a6b86242b903a2c64de3e75bec'

# Initialize Deepgram client using DeepgramClient
dg_client = DeepgramClient(api_key=DEEPGRAM_API_KEY)  # Correct initialization

# File to save the transcription
TEXT_FILE_PATH = "transcription.txt"

# Asynchronous function to transcribe the audio
async def transcribe_audio(filename):
    with open(filename, 'rb') as audio:
        source = {'buffer': audio, 'mimetype': 'audio/wav'}

        # Call Deepgram API for transcription
        response = await dg_client.transcription.prerecorded(source, {'punctuate': True})

        # Extract the transcription
        transcript = response['results']['channels'][0]['alternatives'][0]['transcript']
        print("Transcription: ", transcript)

        # Save transcription to text file
        with open(TEXT_FILE_PATH, 'w') as f:
            f.write(transcript)
        print(f"Transcription saved to {TEXT_FILE_PATH}")

# Run the transcription function
asyncio.run(transcribe_audio(audio_file_path))

RuntimeError: asyncio.run() cannot be called from a running event loop

In [ ]:
import asyncio
from deepgram import DeepgramClient

# Your Deepgram API key
DEEPGRAM_API_KEY = 'a983f9ca4ed091a6b86242b903a2c64de3e75bec'

# Initialize Deepgram client using DeepgramClient
dg_client = DeepgramClient(api_key=DEEPGRAM_API_KEY)  # Correct initialization

# File to save the transcription
TEXT_FILE_PATH = "transcription.txt"

# Asynchronous function to transcribe the audio
async def transcribe_audio(filename):
    with open(filename, 'rb') as audio:
        source = {'buffer': audio, 'mimetype': 'audio/wav'}

        # Call Deepgram API for transcription
        response = await dg_client.transcription.prerecorded(source, {'punctuate': True})

        # Extract the transcription
        transcript = response['results']['channels'][0]['alternatives'][0]['transcript']
        print("Transcription: ", transcript)

        # Save transcription to text file
        with open(TEXT_FILE_PATH, 'w') as f:
            f.write(transcript)
        print(f"Transcription saved to {TEXT_FILE_PATH}")

# Run the transcription function using the existing event loop
# Instead of asyncio.run, use the following:
await transcribe_audio(audio_file_path) # Call the function with await

AttributeError: 'DeepgramClient' object has no attribute 'transcription'

In [ ]:
import asyncio
from deepgram import DeepgramClient

# Your Deepgram API key
DEEPGRAM_API_KEY = 'a983f9ca4ed091a6b86242b903a2c64de3e75bec'

# Initialize Deepgram client using DeepgramClient
dg_client = DeepgramClient(api_key=DEEPGRAM_API_KEY)  # Correct initialization

# File to save the transcription
TEXT_FILE_PATH = "transcription.txt"

# Asynchronous function to transcribe the audio
async def transcribe_audio(filename):
    with open(filename, 'rb') as audio:
        source = {'buffer': audio, 'mimetype': 'audio/wav'}

        # Call Deepgram API for transcription directly using dg_client
        response = await dg_client.prerecorded(source, {'punctuate': True}) # Changed this line

        # Extract the transcription
        transcript = response['results']['channels'][0]['alternatives'][0]['transcript']
        print("Transcription: ", transcript)

        # Save transcription to text file
        with open(TEXT_FILE_PATH, 'w') as f:
            f.write(transcript)
        print(f"Transcription saved to {TEXT_FILE_PATH}")

# Run the transcription function using the existing event loop
# Instead of asyncio.run, use the following:
await transcribe_audio(audio_file_path) # Call the function with await

AttributeError: 'DeepgramClient' object has no attribute 'prerecorded'

In [ ]:
import asyncio
from deepgram import DeepgramClient

# Your Deepgram API key
DEEPGRAM_API_KEY = 'a983f9ca4ed091a6b86242b903a2c64de3e75bec'

# Initialize Deepgram client using DeepgramClient
dg_client = DeepgramClient(api_key=DEEPGRAM_API_KEY)  # Correct initialization

# File to save the transcription
TEXT_FILE_PATH = "transcription.txt"

# Asynchronous function to transcribe the audio
async def transcribe_audio(filename):
    with open(filename, 'rb') as audio:
        source = {'buffer': audio, 'mimetype': 'audio/wav'}

        # Call Deepgram API for transcription using dg_client.transcription.prerecorded
        response = await dg_client.transcription.prerecorded(source, {'punctuate': True})  # Corrected this line

        # Extract the transcription
        transcript = response['results']['channels'][0]['alternatives'][0]['transcript']
        print("Transcription: ", transcript)

        # Save transcription to text file
        with open(TEXT_FILE_PATH, 'w') as f:
            f.write(transcript)
        print(f"Transcription saved to {TEXT_FILE_PATH}")

# Run the transcription function using the existing event loop
# Instead of asyncio.run, use the following:
await transcribe_audio(audio_file_path) # Call the function with await

AttributeError: 'DeepgramClient' object has no attribute 'transcription'

In [ ]:
import asyncio
from deepgram import DeepgramClient

# Your Deepgram API key
DEEPGRAM_API_KEY = 'a983f9ca4ed091a6b86242b903a2c64de3e75bec'

# Initialize Deepgram client using DeepgramClient
dg_client = DeepgramClient(api_key=DEEPGRAM_API_KEY)  # Correct initialization

# File to save the transcription
TEXT_FILE_PATH = "transcription.txt"

# Asynchronous function to transcribe the audio
async def transcribe_audio(filename):
    with open(filename, 'rb') as audio:
        source = {'buffer': audio, 'mimetype': 'audio/wav'}

        # Call Deepgram API for transcription directly using dg_client.prerecorded
        # The 'transcription' attribute is no longer needed in the newer SDK version.
        response = await dg_client.prerecorded(source, {'punctuate': True})

        # Extract the transcription
        transcript = response['results']['channels'][0]['alternatives'][0]['transcript']
        print("Transcription: ", transcript)

        # Save transcription to text file
        with open(TEXT_FILE_PATH, 'w') as f:
            f.write(transcript)
        print(f"Transcription saved to {TEXT_FILE_PATH}")

# Run the transcription function using the existing event loop
# Instead of asyncio.run, use the following:
await transcribe_audio(audio_file_path) # Call the function with await

AttributeError: 'DeepgramClient' object has no attribute 'prerecorded'

In [ ]:
from deepgram import Deepgram
import json

# Your Deepgram API key
DEEPGRAM_API_KEY = 'your_api_key'

# Initialize the Deepgram client
dg_client = Deepgram(DEEPGRAM_API_KEY)

# Function to transcribe the uploaded audio file
def transcribe_audio(file_path):
    with open(file_path, 'rb') as audio:
        source = {'buffer': audio, 'mimetype': 'audio/wav'}  # Adjust the mimetype if necessary

        # Send the audio to Deepgram for transcription
        response = dg_client.transcription.prerecorded(source, {'punctuate': True})

        # Extract the transcription text
        transcript = response['results']['channels'][0]['alternatives'][0]['transcript']
        print("Transcription: ", transcript)

        # Save transcription to a .txt file
        with open("transcription.txt", "w") as f:
            f.write(transcript)
        print("Transcription saved to transcription.txt")

# Transcribe the uploaded audio file
transcribe_audio(audio_file_path)


Exception: 
            FATAL ERROR:
            You are attempting to instantiate a Deepgram object, which is no longer a class in version 3 of this SDK.

            To fix this issue:
                1. You need to revert to the previous version 2 of the SDK: pip install deepgram-sdk==2.12.0
                2. or, update your application's code to use version 3 of this SDK. See the README for more information.

            Things to consider:

                - This Version 3 of the SDK requires Python 3.10 or higher.
                  Older versions (3.9 and lower) of Python are nearing end-of-life: https://devguide.python.org/versions/
                  Understand the risks of using a version of Python nearing EOL.

                - Version 2 of the SDK will receive maintenance updates in the form of security fixes only.
                  No new features will be added to version 2 of the SDK.
            

In [ ]:
import requests

# Your Deepgram API key
DEEPGRAM_API_KEY = 'a983f9ca4ed091a6b86242b903a2c64de3e75bec'

# API Endpoint for transcription
DEEPGRAM_API_URL = "https://api.deepgram.com/v1/listen"

# Transcribe audio function using Deepgram API (HTTP request for version 3)
def transcribe_audio(file_path):
    with open(file_path, 'rb') as audio:
        # HTTP headers
        headers = {
            'Authorization': f'Token {DEEPGRAM_API_KEY}',
            'Content-Type': 'audio/wav',  # Adjust this if your file is not .wav
        }

        # Send the audio to Deepgram's API
        response = requests.post(DEEPGRAM_API_URL, headers=headers, data=audio)

        if response.status_code == 200:
            # Extract the transcription from the response
            transcript = response.json()['results']['channels'][0]['alternatives'][0]['transcript']
            print("Transcription: ", transcript)

            # Save transcription to a .txt file
            with open("transcription.txt", "w") as f:
                f.write(transcript)
            print("Transcription saved to transcription.txt")
        else:
            print(f"Error: {response.status_code}, {response.text}")

# Transcribe the uploaded audio file
transcribe_audio(audio_file_path)


Transcription:  mister quilt is the apostle of the middle classes and we're are glad to welcome his gospel
Transcription saved to transcription.txt


In [ ]:
from google.colab import files

# Download the transcription.txt file
files.download('transcription.txt')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files

# Upload the audio file
uploaded = files.upload()

# The file will be saved with the same name as it is on your local machine.
# You can access it by the key in the `uploaded` dictionary, which is the filename.
audio_file_path = list(uploaded.keys())[0]


Saving Deep_Test.m4a to Deep_Test.m4a


In [ ]:
import requests

# Your Deepgram API key
DEEPGRAM_API_KEY = 'a983f9ca4ed091a6b86242b903a2c64de3e75bec'

# API Endpoint for transcription
DEEPGRAM_API_URL = "https://api.deepgram.com/v1/listen"

# Transcribe audio function using Deepgram API (HTTP request for version 3)
def transcribe_audio(file_path):
    with open(file_path, 'rb') as audio:
        # HTTP headers
        headers = {
            'Authorization': f'Token {DEEPGRAM_API_KEY}',
            'Content-Type': 'audio/wav',  # Adjust this if your file is not .wav
        }

        # Send the audio to Deepgram's API
        response = requests.post(DEEPGRAM_API_URL, headers=headers, data=audio)

        if response.status_code == 200:
            # Extract the transcription from the response
            transcript = response.json()['results']['channels'][0]['alternatives'][0]['transcript']
            print("Transcription: ", transcript)

            # Save transcription to a .txt file
            with open("transcription.txt", "w") as f:
                f.write(transcript)
            print("Transcription saved to transcription.txt")
        else:
            print(f"Error: {response.status_code}, {response.text}")

# Transcribe the uploaded audio file
transcribe_audio(audio_file_path)


Transcription:  hi i'm we are testing the deepgram api please give me the transcription of what i'm talking right now this is for a project in deep learning spearheaded by professor chen there are other team members in this project like s uc ash and bar we are all working on different elements in the project and we are trying to test different api and see if they work fine we are in the initial stages of the project we are trying to figure out things okay
Transcription saved to transcription.txt


In [ ]:
from google.colab import files

# Download the transcription.txt file
files.download('transcription.txt')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>